In [1]:
# Module Imports
from utils import *
from local_files import data_directory

In [2]:
# Constants
movie_data_csv = f"{data_directory}/DataSets/movie.csv"
rating_data_csv = f"{data_directory}/DataSets/rating.csv"
TITLE = 'title'

In [3]:
# Read Data
movie_df = pd.read_csv(movie_data_csv)
rating_df = pd.read_csv(rating_data_csv)

In [4]:
# Data Transformations etc.
users_id = rating_df["userId"].unique()
movies_id = movie_df["movieId"].unique()

In [5]:
movie_df.head(4)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance


In [6]:
item_df = process_item_df(
    item_df=movie_df,
    item_id_col='movieId'
)

KeyError: 'is_genre'

In [ ]:
item_df.head(4)